In [157]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
from tifffile import TiffWriter, TiffFile

In [2]:
a=tf.constant(np.random.rand(4,4), dtype=np.complex128)
b=tf.constant(np.random.rand(4,4), dtype=np.complex128)

In [15]:
phase_factor = 0.124223
def q(u,n):
    u*=tf.exp(phase_factor*1j*tf.complex(n,tf.zeros_like(n)))
    return u

In [36]:
u=tf.constant(np.array([1.2323 + 0.34j, 0.934 + 0.85j], np.complex128))
n=tf.constant(np.array([0.12, 0.09], np.double))
ug = tf.constant(np.array([2 + 1j, 3 - 0.5j], np.complex128))
with tf.GradientTape() as g:
    g.watch(u)
    g.watch(n)
    unew=q(u,n)
    l=tf.tensordot(unew,tf.math.conj(ug),1)
g.gradient(l,[u,n])

[<tf.Tensor: shape=(2,), dtype=complex128, numpy=array([2.014684  +0.97007648j, 2.99422259-0.53350826j])>,
 <tf.Tensor: shape=(2,), dtype=float64, numpy=array([ 0.06340748, -0.37805853])>]

In [16]:
q(u,n)

<tf.Tensor: shape=(2,), dtype=complex128, numpy=array([1.22709498+0.35833114j, 0.92443877+0.86038885j])>

In [54]:
np.random.random((4,4))

array([[0.94652344, 0.75778901, 0.36268692, 0.17039358],
       [0.6218963 , 0.77670778, 0.81780564, 0.67399338],
       [0.41853109, 0.2668255 , 0.29332194, 0.00581977],
       [0.65006947, 0.21395241, 0.8160816 , 0.928916  ]])

In [137]:
u=np.array([[0.09562465, 0.00480245, 0.15706965, 0.86148982],
       [0.06099826, 0.19465191, 0.99168264, 0.06427324],
       [0.13013975, 0.16247964, 0.21700103, 0.3390049 ],
       [0.69911775, 0.00399147, 0.8371409 , 0.81915701]])+np.array([[0.36790823, 0.96613423, 0.57488218, 0.78752347],
       [0.30869711, 0.16676483, 0.99859666, 0.54262361],
       [0.10260755, 0.93277456, 0.50749209, 0.48005492],
       [0.67407622, 0.33936744, 0.23437531, 0.83429173]])*1j
u=u*0.1+0.95
n=np.array([[0.94652344, 0.75778901, 0.36268692, 0.17039358],
       [0.6218963 , 0.77670778, 0.81780564, 0.67399338],
       [0.41853109, 0.2668255 , 0.29332194, 0.00581977],
       [0.65006947, 0.21395241, 0.8160816 , 0.928916  ]])*0.1
# u=np.ones_like(u)
u, n=[tf.constant(i, dtype=tf.complex128) for i in (u,n)]

In [186]:
u=tf.ones((128, 128), dtype=tf.complex128)
with tf.GradientTape() as g:
    g.watch(n)
    # nc=tf.complex(n,tf.zeros_like(n))
    for ni in n:
        u=bpm_step(u,1,ni)
    l=tf.reduce_sum(tf.square(tf.abs(u)-mea))
print(l)
ng=g.gradient(l, n)
# unew=bpm_step(u,2,n-0.01*tf.cast(tf.math.real(ng),tf.complex128))
# l=tf.reduce_sum(tf.square(tf.abs(unew-0.3)))
# print(l)

tf.Tensor(13.503490752549002, shape=(), dtype=float64)


In [191]:
ng_cuda=np.load(r"D:\Users\zjb\Documents\School Tasks\data\ssnp\ng.npy")

In [197]:
np.sum(np.abs(np.real(ng.numpy())-ng_cuda))

4.999428124252148e-09

In [183]:
mea = tiff_import(r"D:\Users\zjb\Documents\School Tasks\data\ssnp\bpmtest.tiff", dtype=tf.float64)*2
n = tiff_import(r"D:\Users\zjb\Documents\School Tasks\data\ssnp\ball.tiff")*0.0

In [168]:
DATA_TYPE=tf.complex128
SIZE=[128,128]
RES=[0.1, 0.1, 0.1]
EPS=1E-8
N0=1
def tiff_import(path: str, dtype=DATA_TYPE):
    with TiffFile(path) as file:
        img = np.squeeze(np.stack([page.asarray() for page in file.pages]))
    if img.dtype.type == np.uint16:
        img = img.astype(np.double) / 65535
    elif img.dtype.type == np.uint8:
        warn("Importing uint8 image, please use uint16 if possible")
        img = img.astype(np.double) / 255
    return tf.constant(img, dtype)

@tf.function
def bpm_step(u, dz, n=None):
    """
    BPM main operation of one step

    :param u: x-y complex amplitude
    :param dz: step size along z axis
    :param n: refractive index along x-y distribution in this slice. Use background N0 if not provided.
    :return: new (u, u_d) after a step towards +z direction
    """
    for t_in in (u, n):
        if t_in is not None:
            if t_in.shape != SIZE[:2]:
                raise ValueError(f"the x,y shape {t_in.shape} is not {SIZE[:2]}")

    a = tf.signal.fft2d(u)
    evan_rm = tf.constant(_evanescent_absorb(), DATA_TYPE)
    a *= evan_rm
    u = tf.signal.ifft2d(tf.exp(_kz() * (1j * dz)) * a)
    if n is not None:
        u *= tf.exp(n * (1j * (2 * np.pi * RES[2] * N0) * dz))
    return u
def _c_gamma():
    """
    Calculate cos(gamma) as a constant array at frequency domain. Gamma is the angle
    between the wave vector and z-axis. Note: N.A.=sin(gamma)

    The array is pre-shifted for later FFT operation.

    :return: cos(gamma) numpy array
    """
    c_alpha, c_beta = [
        np.fft.ifftshift(np.arange(-SIZE[i] / 2, SIZE[i] / 2).astype(np.double)) / SIZE[i] / RES[i]
        for i in (0, 1)
    ]
    c_gamma = np.sqrt(np.maximum(1 - (np.square(c_alpha) + np.square(c_beta[:, None])), EPS))
    return c_gamma

def _kz():
    kz = tf.constant(_c_gamma() * (2 * np.pi * RES[2] * N0), DATA_TYPE)
    return kz

def _evanescent_absorb() -> np.ndarray:
    """
        For 0.98<N.A.<1, decrease to 1~1/e per step

        For N.A.>1 decrease to 1/e per step

        :return: a 2D angular spectrum transmittance numpy array
    """
    return np.exp(np.minimum((_c_gamma() - 0.2) * 5, 0))